In [8]:
import torch
from ultralytics import YOLO
import cv2
import glob
import os
import shutil
import numpy as np

def dev():
    if torch.cuda.is_available() is True: 
        device = "cuda"    
    else:
        if torch.backends.mps.is_available() is True:device = "mps"
        else: 
            device = "cpu"
    return device
device = dev()
if device == 'mps':
    torch.mps.empty_cache()
print(device)


mps


In [48]:
def train_find (data, name_project): #обучение YOLO_find
    model_yolo_find= YOLO('yolov8s.pt')
    model_yolo_find.to(device)
    model_yolo_find.train(data=data, 
                batch = 4,
                epochs= 50, 
                project=name_project, # Папка для сохранения результатов
                name='custom_run', 
                augment=True,
                conf =  0.55,
                iou = 0.4,
                imgsz=1536)
    # Перемещаем файл best.pt в нужное место
    c_r =  f"{name_project}"
    files = [f for f in os.listdir(c_r)]
    last_dir = sorted(files)[-1]
    orig_w = f'{name_project}/{last_dir}/weights/best.pt'
    new_w = f'{name_project}.pt'
    shutil.copy(orig_w, new_w)
    print('Обучение завершено!')

In [2]:
def find (path, model_name):# вырезка сегмета
    model_find = YOLO(model_name)  
    # Папка с изображениями
    image_folder = f'{path}/images/Test'  # Путь к папке с изображениями
    output_folder_seg = f'{path}/output_seg/'  # Папка для сохранения вырезанных участко
    os.makedirs(output_folder_seg, exist_ok=True)  # Создаем папку, если её нет

    # Обработка каждого изображения
    image_paths = glob.glob(f'{image_folder}/*.jpg')  # Все файлы с расширением .
    for image_path in image_paths:
        # Загрузка изображения
        image = cv2.imread(image_path)
        original_name = os.path.splitext(os.path.basename(image_path))[0]  # Имя файла без расширения
        
        # Выполнение предсказаний
        results = model_find(image_path)
        
        # Получение предсказанных объектов
        detections = results[0].boxes.xyxy  # Координаты ограничивающих рамок (xmin, ymin, xmax, ymax)
        scores = results[0].boxes.conf      # Уверенность предсказания
        classes = results[0].boxes.cls      # Предсказанные классы
        
        # Счетчик для уникального имени каждого участка
        counter = 0
        
        # Вырезка и сохранение обнаруженных участков
        for box, score, cls in zip(detections, scores, classes):
            x1, y1, x2, y2 = map(int, box)  # Преобразуем координаты в целые числа
            
            # Вырезаем обнаруженную область
            cropped_image = image[y1:y2, x1:x2]
            
            # Проверяем, что область не пуста
            if cropped_image.size > 0:
                # Сохраняем вырезанную область
                output_path = os.path.join(output_folder_seg, f"{original_name}_object{counter}.jpg")
                cv2.imwrite(output_path, cropped_image)
                print(f"Сохранён объект: {output_path}")
                counter += 1


    print("Обработка всех изображений завершена!")

In [50]:
def train_class (data, classify_project): #обучение классификатора
    model_yolo_class= YOLO("yolov8s-cls.pt")  # Можно заменить на yolov8s-cls.pt, yolov8m-cls.pt и т.д.
    model_yolo_class.to(device)
    # Запускаем обучение
    model_yolo_class.train(
        data=data,   # Путь к папке с train/ и val/
        epochs=50,        # Количество эпох
        imgsz=224,        # Размер входного изображения
        batch=16,         # Размер батча
        lr0=0.0001,        # Начальная скорость обучения
        augment=True, 
        project=classify_project,
        name="custom_run")
    c_r =  f"{classify_project}"
    files = [f for f in os.listdir(c_r)]
    last_dir = sorted(files)[-1]
    orig_w = f'{classify_project}/{last_dir}/weights/best.pt'
    new_w = f'{classify_project}.pt'
    shutil.copy(orig_w, new_w)
    print('Обучение завершено!')

In [3]:
def classif (model_name_class, data): #классификация
    model_yolo_class = YOLO(model_name_class)  # Укажи путь к лучшим весам
    # Папка с изображениями для классификации
    input_folder = data  # Замените на путь к вашим тестовым изображениям
    # Папка для сортированных результатов
    output_folder = "sorted_results"
    os.makedirs(output_folder, exist_ok=True)

    # Проходим по всем изображениям во входной папке
    for fname in os.listdir(input_folder):
        if not fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue  # Пропускаем файлы неподходящих форматов
        
        image_path = os.path.join(input_folder, fname)
        
        # Получаем предсказание модели для данного изображения
        # predict возвращает список Prediction объектов
        results = model_yolo_class.predict(source=image_path, verbose=False)
        
        if results:
            # Для классификации предполагается, что результат содержит вероятность для каждого класса.
            # Здесь мы берём первую Prediction и извлекаем вероятности.
            probs = results[0].probs.data  # Преобразуем в numpy-массив
            predicted_idx = np.argmax(np.array(probs))
            predicted_label = model_yolo_class.names.get(predicted_idx, "unknown")
        else:
            predicted_label = "unknown"
        
        # Создаем папку для этого класса, если ее еще нет
        target_folder = os.path.join(output_folder, predicted_label)
        os.makedirs(target_folder, exist_ok=True)
        
        # Перемещаем (или копируем) изображение в соответствующую папку
        shutil.copy(image_path, os.path.join(target_folder, fname))
        print(f"Изображение '{fname}' классифицировано как '{predicted_label}' и скопированно в '{target_folder}'.")

    print("Сортировка изображений завершена!")

In [4]:
#куча переменных для удобства
vortex = 'find_vort.yaml'
vortex_p = 'data_find_vort'
proj_vort = 'find_vortex'
proj_sort_vort = 'sorted_vortex'
micron = 'find_mic.yaml'
micron_p = 'data_find_mic'
proj_mic = 'find_micron'
proj_sort_mic = 'sorted_micron'
pt='.pt'
outseg = 'output_seg'

In [54]:
train_find(vortex, proj_vort)

New https://pypi.org/project/ultralytics/8.3.99 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=find_vort.yaml, epochs=50, time=None, patience=100, batch=4, imgsz=1536, save=True, save_period=-1, cache=False, device=mps:0, workers=8, project=find_vortex, name=custom_run, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.55, iou=0.4, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

Fontconfig warning: ignoring UTF-8: not a valid region tag


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

train: Scanning data_find_vort/labels/Train.cache... 128 images, 0 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

      50/50      12.9G     0.6054     0.3311     0.7805          4       1536: 100%|██████████| 32/32 [01:05<00:00,  2.04s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:19<00:00,  1.20s/it]

                   all        128        128          1          1      0.995      0.983



50 epochs completed in 1.212 hours.
Optimizer stripped from find_vortex/custom_run/weights/last.pt, 22.8MB
Optimizer stripped from find_vortex/custom_run/weights/best.pt, 22.8MB

Validating find_vortex/custom_run/weights/best.pt...
Ultralytics 8.3.39 🚀 Python-3.11.7 torch-2.4.1 MPS (Apple M3 Pro)
Model summary (fused): 168 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:34<00:00,  2.17s/it]


                   all        128        128          1          1      0.995      0.984
Speed: 1.9ms preprocess, 240.7ms inference, 0.0ms loss, 9.0ms postprocess per image
Results saved to find_vortex/custom_run
Обучение завершено!


In [56]:
train_find(micron, proj_mic)

New https://pypi.org/project/ultralytics/8.3.99 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=find_mic.yaml, epochs=50, time=None, patience=100, batch=4, imgsz=1536, save=True, save_period=-1, cache=False, device=mps:0, workers=8, project=find_micron, name=custom_run, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.55, iou=0.4, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_con

train: Scanning data_find_mic/labels/Train.cache... 164 images, 0 backgrounds, 0 corrupt: 100%|██████████| 164/164 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning data_find_mic/labels/Train.cache... 164 images, 0 backgrounds, 0 corrupt: 100%|██████████| 164/164 [00:00<?, ?it/s]

Plotting labels to find_micron/custom_run/labels.jpg... 


...

      50/50      16.8G     0.4067     0.2272     0.9146          4       1536: 100%|██████████| 41/41 [02:35<00:00,  3.79s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:19<00:00,  1.08it/s]

                   all        164        164          1          1      0.995      0.936



50 epochs completed in 2.360 hours.
Optimizer stripped from find_micron/custom_run/weights/last.pt, 22.8MB
Optimizer stripped from find_micron/custom_run/weights/best.pt, 22.8MB

Validating find_micron/custom_run/weights/best.pt...
Ultralytics 8.3.39 🚀 Python-3.11.7 torch-2.4.1 MPS (Apple M3 Pro)
Model summary (fused): 168 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:50<00:00,  2.43s/it]


                   all        164        164          1          1      0.995      0.933
Speed: 1.7ms preprocess, 284.2ms inference, 0.0ms loss, 8.3ms postprocess per image
Results saved to find_micron/custom_run
Обучение завершено!


In [5]:
find(vortex_p,proj_vort+pt)
find(micron_p,proj_mic+pt)


image 1/1 /Users/kujibka/Desktop/diplom/data_find_vort/images/Test/vort_92.jpg: 1536x1536 1 vort, 570.3ms
Speed: 5.7ms preprocess, 570.3ms inference, 1.3ms postprocess per image at shape (1, 3, 1536, 1536)
Сохранён объект: data_find_vort/output_seg/vort_92_object0.jpg

image 1/1 /Users/kujibka/Desktop/diplom/data_find_vort/images/Test/vort_86.jpg: 1536x1536 1 vort, 529.7ms
Speed: 5.0ms preprocess, 529.7ms inference, 0.4ms postprocess per image at shape (1, 3, 1536, 1536)
Сохранён объект: data_find_vort/output_seg/vort_86_object0.jpg

image 1/1 /Users/kujibka/Desktop/diplom/data_find_vort/images/Test/vort_87.jpg: 1536x1536 1 vort, 517.4ms
Speed: 4.8ms preprocess, 517.4ms inference, 0.4ms postprocess per image at shape (1, 3, 1536, 1536)
Сохранён объект: data_find_vort/output_seg/vort_87_object0.jpg

image 1/1 /Users/kujibka/Desktop/diplom/data_find_vort/images/Test/vort_93.jpg: 1536x1536 1 vort, 516.5ms
Speed: 5.1ms preprocess, 516.5ms inference, 0.4ms postprocess per image at shape (1

In [68]:
train_class(f'{vortex_p}/sort_vort',proj_sort_vort)

100%|██████████| 12.3M/12.3M [00:01<00:00, 6.98MB/s]


New https://pypi.org/project/ultralytics/8.3.99 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=classify, mode=train, model=yolov8s-cls.pt, data=data_find_vort/sort_vort, epochs=50, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=mps:0, workers=8, project=sorted_vortex, name=custom_run, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_la

train: Scanning /Users/kujibka/Desktop/diplom/data_find_vort/sort_vort/train... 120 images, 0 corrupt: 100%|██████████| 120/120 [00:00<00:00, 4451.92it/s]

train: New cache created: /Users/kujibka/Desktop/diplom/data_find_vort/sort_vort/train.cache



val: Scanning /Users/kujibka/Desktop/diplom/data_find_vort/sort_vort/val... 25 images, 0 corrupt: 100%|██████████| 25/25 [00:00<00:00, 4054.50it/s]

...


50 epochs completed in 0.034 hours.
Optimizer stripped from sorted_vortex/custom_run/weights/last.pt, 10.3MB
Optimizer stripped from sorted_vortex/custom_run/weights/best.pt, 10.3MB

Validating sorted_vortex/custom_run/weights/best.pt...
Ultralytics 8.3.39 🚀 Python-3.11.7 torch-2.4.1 MPS (Apple M3 Pro)
YOLOv8s-cls summary (fused): 73 layers, 5,077,762 parameters, 0 gradients, 12.4 GFLOPs
train: /Users/kujibka/Desktop/diplom/data_find_vort/sort_vort/train... found 120 images in 2 classes ✅ 
val: /Users/kujibka/Desktop/diplom/data_find_vort/sort_vort/val... found 25 images in 2 classes ✅ 
test: /Users/kujibka/Desktop/diplom/data_find_vort/sort_vort/test... found 145 images in 1 classes: ERROR ❌️ requires 2 classes, not 1


               classes   top1_acc   top5_acc:   0%|          | 0/1 [00:00<?, ?it/s]

WARNING ⚠️ ClassificationModel does not support 'augment=True' prediction. Reverting to single-scale prediction.


               classes   top1_acc   top5_acc: 100%|██████████| 1/1 [00:01<00:00,  1.92s/it]


                   all       0.88          1
Speed: 0.9ms preprocess, 66.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to sorted_vortex/custom_run
Обучение завершено!


In [69]:
train_class(f'{micron_p}/sort_mic',proj_sort_mic)

New https://pypi.org/project/ultralytics/8.3.99 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=classify, mode=train, model=yolov8s-cls.pt, data=data_find_mic/sort_mic, epochs=50, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=mps:0, workers=8, project=sorted_micron, name=custom_run, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labe

train: Scanning /Users/kujibka/Desktop/diplom/data_find_mic/sort_mic/train... 99 images, 0 corrupt: 100%|██████████| 99/99 [00:00<00:00, 4883.58it/s]

train: New cache created: /Users/kujibka/Desktop/diplom/data_find_mic/sort_mic/train.cache



val: Scanning /Users/kujibka/Desktop/diplom/data_find_mic/sort_mic/val... 27 images, 0 corrupt: 100%|██████████| 27/27 [00:00<00:00, 5092.46it/s]

val: New cache created: /Users/kujibka/Desktop/diplom/data_find_mic/sort_mic/val.cache
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.0001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)


...


50 epochs completed in 0.017 hours.
Optimizer stripped from sorted_micron/custom_run/weights/last.pt, 10.3MB
Optimizer stripped from sorted_micron/custom_run/weights/best.pt, 10.3MB

Validating sorted_micron/custom_run/weights/best.pt...
Ultralytics 8.3.39 🚀 Python-3.11.7 torch-2.4.1 MPS (Apple M3 Pro)
YOLOv8s-cls summary (fused): 73 layers, 5,077,762 parameters, 0 gradients, 12.4 GFLOPs
train: /Users/kujibka/Desktop/diplom/data_find_mic/sort_mic/train... found 99 images in 2 classes ✅ 
val: /Users/kujibka/Desktop/diplom/data_find_mic/sort_mic/val... found 27 images in 2 classes ✅ 
test: /Users/kujibka/Desktop/diplom/data_find_mic/sort_mic/test... found 165 images in 1 classes: ERROR ❌️ requires 2 classes, not 1


               classes   top1_acc   top5_acc:   0%|          | 0/1 [00:00<?, ?it/s]

WARNING ⚠️ ClassificationModel does not support 'augment=True' prediction. Reverting to single-scale prediction.


               classes   top1_acc   top5_acc: 100%|██████████| 1/1 [00:01<00:00,  1.72s/it]


                   all      0.815          1
Speed: 0.1ms preprocess, 59.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to sorted_micron/custom_run
Обучение завершено!


In [9]:
classif(proj_sort_vort+pt, vortex_p+'/'+outseg)
classif(proj_sort_mic+pt, micron_p+'/'+outseg)

Изображение 'vort_92_object0.jpg' классифицировано как 'bad' и скопированно в 'sorted_results/bad'.
Изображение 'vort_106_object0.jpg' классифицировано как 'good' и скопированно в 'sorted_results/good'.
Изображение 'vort_113_object0.jpg' классифицировано как 'bad' и скопированно в 'sorted_results/bad'.
Изображение 'vort_87_object0.jpg' классифицировано как 'good' и скопированно в 'sorted_results/good'.
Изображение 'wqe4_object0.jpg' классифицировано как 'good' и скопированно в 'sorted_results/good'.
Изображение 'vort_97_object0.jpg' классифицировано как 'good' и скопированно в 'sorted_results/good'.
Изображение 'vort_103_object0.jpg' классифицировано как 'good' и скопированно в 'sorted_results/good'.
Изображение 'vort_88_object0.jpg' классифицировано как 'good' и скопированно в 'sorted_results/good'.
Изображение 'vort_100_object0.jpg' классифицировано как 'good' и скопированно в 'sorted_results/good'.
Изображение 'vort_94_object0.jpg' классифицировано как 'good' и скопированно в 'sorte